# Datamodel Conditions: AI-Generated Scripts for M/TEXT

Generate Rhino JavaScript code for `<Script>` tags in M/TEXT BusinessData.datamodel nodes.

## M/TEXT Context

In M/TEXT, **BusinessData.datamodel** nodes can contain `<Script>` tags that define:
- **Calculated fields**: Compute values from other fields (e.g., FullName = Firstname + Lastname)
- **Validation logic**: Check field values before document generation  
- **Conditional values**: Set field values based on business rules

**Traditional Approach:** Hand-write Rhino JavaScript (ES5-compatible) for each field  
**AI Approach:** Describe the logic in plain text, LLM generates the JavaScript

### Rhino JavaScript Constraints

M/TEXT uses **Mozilla Rhino 1.8** as its JavaScript engine, which only supports:
- ES5 features (no ES6+, no arrow functions, no template literals)
- `var` declarations only (no `let`/`const`)
- String concatenation with `+` (no template literals)
- Access to other fields via `$FieldName` syntax

## Who This Is For

**M/TEXT Developers**: Automate writing datamodel scripts without memorizing Rhino's limited syntax.

**Business Analysts**: Define business logic in plain English, let the LLM handle the code.

This notebook shows a portable pattern that works with any LLM provider.



## How It Works

The workflow follows this sequence:

1. **Input Validation**: Receive a datamodel XML with `<Node>` elements containing empty `<Script/>` tags and condition descriptions
2. **Prompt Composition**: Create system prompt defining Rhino JavaScript constraints and JSON output format
3. **LLM Generation**: Call Claude Sonnet 4 to generate minimal JavaScript for each field requiring logic
4. **JSON Extraction**: Parse response to get `{NodeName: "script code"}` mappings
5. **XML Injection**: Insert generated scripts into corresponding `<Script>` tags in the datamodel
6. **Response**: Return updated datamodel XML with populated scripts

The entire flow typically completes in 8-15 seconds depending on complexity.


## Inputs and Requirements

**Required:**
- **Datamodel XML**: Schema with `<Node>` elements containing empty `<Script/>` tags within `<Settings>`
- **Condition Descriptions**: Free-text explaining what logic each field should compute or validate

**Example Datamodel:**
```xml
<DataModel>
  <Node name="FullName" type="String">
    <Settings><Script/></Settings>
  </Node>
  <Node name="Firstname" type="String"/>
  <Node name="Lastname" type="String"/>
</DataModel>
```

**Example Descriptions:**
```
- FullName: Concatenate Firstname and Lastname with a space
- Age: Calculate years from BirthDate to today
- IsValid: Return true if Email contains '@'
```

Scripts are constrained to **Rhino JavaScript subset** (no ES6+ features, no arrow functions, no template literals).


In [ ]:
# Example inputs for the conditions workflow

DATAMODEL_XML = """
<DataModel>
  <Node name="FullName" type="String">
    <Settings><Script/></Settings>
  </Node>
  <Node name="Firstname" type="String"/>
  <Node name="Lastname" type="String"/>
  <Node name="Age" type="Integer">
    <Settings><Script/></Settings>
  </Node>
  <Node name="BirthDate" type="Date"/>
</DataModel>
""".strip()

DESCRIPTIONS = """
- FullName: Concatenate Firstname and Lastname with a space between them
- Age: Calculate age in years from BirthDate to current date
""".strip()



## System Prompt: The Core of the Pattern

This system prompt teaches the LLM about Rhino's constraints and M/TEXT's `$FieldName` syntax.

### M/TEXT-Specific Rules

- **Output format**: JSON mapping `{NodeName: "javascript code"}`
- **Variable access**: Use `$FieldName` to reference other datamodel nodes
- **End-of-script rule**: Last line must be the variable name alone (Rhino expects this)
- **No return statement**: Scripts evaluate to the last expression

### Rhino Constraints

- Use `var` only (no `let`/`const`)
- String concatenation with `+` (no template literals)
- No ES6+ features (no arrow functions, no array methods, no destructuring)
- Date handling: `.valueOf().getTime()` for millisecond math

### The Prompt 


In [ ]:
SYSTEM = """
You receive a datamodel XML containing many <Node> elements with a nested <Settings> that includes a <Script/> tag.

Task: Analyze the conditions and return ONLY the JavaScript code for each applicable Node's <Script> tag.

STRICT OUTPUT CONTRACT:
Return a JSON object where:
- Keys are the Node names that need script modifications
- Values are the JavaScript code to insert into their <Script> tags
- If no nodes need modification, return an empty object {}

JavaScript constraints (Rhino-like engine):
- Use only basic features: 'var' declarations, if/else, string concatenation with '+', basic math
- NO ternary operators, NO arrow functions, NO template literals, NO array methods, NO destructuring, NO async/await, NO try/catch
- Always declare variables with 'var'
- End-of-script rule: Do NOT use 'return'. The FINAL LINE MUST be the variable name by itself (e.g., sRetVal;)
- Access values using $<NodeName> (e.g., $Firstname, $Lastname)
- For date values, prefer .valueOf().getTime() and do basic math in milliseconds

Simplicity rules (very important):
- Do NOT add defensive checks for null/empty/missing values unless explicitly requested
- Keep scripts minimal and direct to produce the required output; avoid extra branches or guards
- Prefer straight-line code over cascading conditionals when possible

Examples of concise, acceptable scripts:
- Concatenation (multiline):
  var sRetVal = "";
  
  sRetVal = $Firstname + " " + $Lastname;
  
  sRetVal;

- Date diff (days):
  var d1 = $StartDate.valueOf().getTime();
  var d2 = $EndDate.valueOf().getTime();
  var days = (d2 - d1) / (1000 * 60 * 60 * 24);
  
  days;

Output ONLY valid JSON. NO markdown fences, NO prose, NO explanations.
"""



## User Prompt (Inputs + Task)

The user prompt contains the datamodel XML and the condition descriptions. The LLM returns JSON mapping node names to their JavaScript code.


In [ ]:
USER = f"""
Datamodel (XML):
{DATAMODEL_XML}

Condition Descriptions:
{DESCRIPTIONS}

Return ONLY a JSON object mapping Node names to concise JavaScript code to place inside their <Script> tags. Keep scripts minimal; avoid unnecessary null/empty checks. NO markdown or commentary.
""".strip()

# Example expected response:
# {
#   "FullName": "var sRetVal = \"\";\n\nsRetVal = $Firstname + \" \" + $Lastname;\n\nsRetVal;",
#   "Age": "var d1 = $BirthDate.valueOf().getTime();\nvar now = new Date().getTime();\nvar years = (now - d1) / (1000 * 60 * 60 * 24 * 365.25);\n\nyears;"
# }



## Example Implementation (Python)

Here's how to implement this pattern with Anthropic or OpenAI:

```python
# Example with Anthropic
from anthropic import Anthropic
import os, json

client = Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))

def generate_datamodel_scripts(datamodel_xml: str, descriptions: str):
    """Generate Rhino JavaScript scripts for datamodel nodes"""
    user_prompt = f"""Datamodel (XML):
{datamodel_xml}

Condition Descriptions:
{descriptions}

Return ONLY a JSON object mapping Node names to concise JavaScript code to place inside their <Script> tags. Keep scripts minimal; avoid unnecessary null/empty checks. NO markdown or commentary."""
    
    response = client.messages.create(
        model="claude-sonnet-4-0",
        max_tokens=4000,
        messages=[{"role": "user", "content": SYSTEM + "\n\n" + user_prompt}]
    )
    
    # Extract JSON from response
    raw = response.content[0].text
    scripts_json = json.loads(raw)
    
    return scripts_json

# Usage:
# scripts = generate_datamodel_scripts(DATAMODEL_XML, DESCRIPTIONS)
# Then inject into datamodel XML (see next section)
```


## Script Injection Pattern

After the LLM generates the JSON mapping, the scripts are injected into the datamodel XML:

1. **Extract JSON**: Parse the LLM response to get `{NodeName: "script code"}` object
2. **Find nodes**: For each key in the JSON, locate the corresponding `<Node name="...">` in the datamodel
3. **Insert script**: Replace the empty `<Script/>` tag with `<Script>GENERATED_CODE</Script>`
4. **Escape XML**: The JavaScript code is XML-escaped (e.g., `<` becomes `&lt;`)

### Example

**Input datamodel:**
```xml
<Node name="FullName" type="String">
  <Settings><Script/></Settings>
</Node>
```

**LLM generates:**
```json
{
  "FullName": "var sRetVal = \"\";\n\nsRetVal = $Firstname + \" \" + $Lastname;\n\nsRetVal;"
}
```

**Output datamodel:**
```xml
<Node name="FullName" type="String">
  <Settings><Script>var sRetVal = "";

sRetVal = $Firstname + " " + $Lastname;

sRetVal;</Script></Settings>
</Node>
```


## Integration with M/TEXT

Once scripts are generated and injected:

1. **Load in M/TEXT Designer**: Import the updated datamodel XML
2. **Test with sample data**: Use M/TEXT's datamodel testing tools to verify script logic
3. **Verify field access**: Ensure `$FieldName` references resolve correctly in your datamodel
4. **Check Rhino compatibility**: Scripts execute in Rhino 1.8 environment

### Script Execution Context

- Scripts execute when the field value is needed during document generation
- Access to other fields via `$NodeName` syntax
- Final line of script determines the field's value
- Scripts can reference fields at any level in the datamodel hierarchy
